In [1]:
import pandas as pd
import os
import pickle

### getting the file lists

In [2]:
# setting the data directory
data_dir = '../../AI_perception_survey_data/WE'
save_dir = '../../AI_perception_survey_data/WE/cleaned'

In [3]:
# getting all file names
all_files = os.listdir(data_dir)

In [4]:
all_files

['Public Perception of Artificial Intelligence Technologies [UK].csv',
 '.DS_Store',
 'Public Perception of Artificial Intelligence Technologies [Deutsche].csv',
 'cleaned',
 'Pollfish_Survey_%C3%96ffentliche_Wahrnehmung_der_k%C3%BCnstlichen_Intelligenz_%5BDeutsche%5D_1979903.xlsx',
 'Public Perception of Artificial Intelligence Technologies [intl_Ver].csv',
 "Perception Publique des technologies de l'Intelligence Artificielle [FR].csv",
 'Pollfish_Survey_Perception_Publique_des_technologies_de_l%27Intelligence_Artificielle_%5BFR%5D_1979900.xlsx']

In [5]:
# extracting the pollfish ones
pf_files = [file for file in all_files if file.split('_')[0] == 'Pollfish']

In [6]:
pf_files

['Pollfish_Survey_%C3%96ffentliche_Wahrnehmung_der_k%C3%BCnstlichen_Intelligenz_%5BDeutsche%5D_1979903.xlsx',
 'Pollfish_Survey_Perception_Publique_des_technologies_de_l%27Intelligence_Artificielle_%5BFR%5D_1979900.xlsx']

In [7]:
# loading all files in
pf_de = pd.read_excel(os.path.join(data_dir, pf_files[0]), sheet_name = 'Individuals')
pf_fr = pd.read_excel(os.path.join(data_dir, pf_files[1]), sheet_name = 'Individuals')

In [8]:
# taking a brief look
pf_fr.head()

,ID,Time Started,Time Finished,Manufacturer,OS,Country,Area,City,Provider,Gender,...,Gamer,Value Shopper,Food and Dining Lover,Entertainment Enthusiast,Fashionista,Job Seeker,Insurance,Real Estate,Car Purchase,Unnamed: 65
0,467934861,2019-05-17 08:12:35,2019-05-17 08:17:29,OnePlus,Android,FR,Brittany,Miniac-Morvan,Bouygues,male,...,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN
1,-1953996334,2019-05-17 08:25:55,2019-05-17 08:27:54,Huawei,Android,FR,Île-de-France,Paris,Bouygues,male,...,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
2,873807755,2019-05-17 08:16:32,2019-05-17 08:22:48,Unknown,Android,FR,Nouvelle-Aquitaine,Agen,SFR,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1008838748,2019-05-17 08:22:18,2019-05-17 08:26:14,Unknown,Web,FR,Île-de-France,Paris,NaN,female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-920061686,2019-05-17 08:12:58,2019-05-17 08:14:22,Unknown,iOS,BE,Flanders,Ghent,NaN,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have a lot of extra cols that might be interesting in other cases but for reason of uniformity we'll not need them for this analysis

### cleaning, renaming and reordering columns

In [9]:
# seeing which cols to keep
for i, col in enumerate(pf_fr.columns):
    print('*******************', i, '*******************')
    print(col)
    print(' ')

******************* 0 *******************
ID
 
******************* 1 *******************
Time Started
 
******************* 2 *******************
Time Finished
 
******************* 3 *******************
Manufacturer
 
******************* 4 *******************
OS
 
******************* 5 *******************
Country
 
******************* 6 *******************
Area
 
******************* 7 *******************
City
 
******************* 8 *******************
Provider
 
******************* 9 *******************
Gender
 
******************* 10 *******************
Age
 
******************* 11 *******************
Year Of Birth
 
******************* 12 *******************
Comment évalueriez-vous votre connaissance personnelle des technologies d'intelligence artificielle ?
 
******************* 13 *******************
Parmi les affirmations suivantes, lesquelles sont * fausses * concernant la nature ou les capacités de l'intelligence artificielle ?
 
******************* 14 *******************
Avez

In [10]:
# selecting which cols to keep
cols_to_keep_fr = list(pf_fr.columns[12:37]) + [pf_fr.columns[5]] + [pf_fr.columns[10]] + [pf_fr.columns[39]]
cols_to_keep_de = list(pf_de.columns[12:37]) + [pf_de.columns[5]] + [pf_de.columns[10]] + [pf_fr.columns[39]]

In [11]:
# slimming the dfs down
pf_fr = pf_fr[cols_to_keep_fr]
pf_de = pf_de[cols_to_keep_de]

In [12]:
# new col names
cols_renamed = ['q{:02}'.format(i+1) for i in range(21)] + ['q{:02}'.format(i) for i in range(26,30)] + ['country_of_residence', 'q25_sup', 'q27_sup']

In [13]:
# renaming cols
pf_fr.columns = cols_renamed
pf_de.columns = cols_renamed

### recoding answers

In [14]:
pf_fr.head(3)

,q01,q02,q03,q04,q05,q06,q07,q08,q09,q10,...,q19,q20,q21,q26,q27,q28,q29,country_of_residence,q25_sup,q27_sup
0,J'en sais beaucoup sur le sujet,L’architecture du réseau neuronal récurrent ri...,Oui,5,6,5,7,5,7,Nouvelles opportunités commerciales | Meilleur...,...,Les compétences requises pour le travail sont ...,Oui,L'incapacité de l'IA à comprendre les nuances ...,Ingénieur métallurgique,Bachelor/BTS ou équivalent,Oui,6,FR,18 - 24,high_school
1,J'en sais beaucoup sur le sujet,L’IA peut rédiger des articles de qualité huma...,Oui,7,7,6,7,7,7,Nouvelles opportunités commerciales | Augmenta...,...,Les compétences requises pour le travail sont ...,Oui,L'incapacité de l'IA à comprendre les nuances ...,Ingénieur,"Doctorat, son équivalent ou supérieur",Oui,7,FR,18 - 24,high_school
2,J'en ai entendu parler mais je ne suis pas sur...,L'IA n'est capable d'effectuer qu'une gamme li...,Peut-être,4,6,4,6,4,6,aucun,...,Manque de créativité,Peut-être,L'IA peut être biaisée et renforcer l'injustic...,Sans profession,Diplôme d'études secondaires ou l'équivalent,Oui,5,FR,45 - 54,vocational_technical_college


The seperators are '|', whereas our recoding function uses ';' as the default seperator, we need to replace

In [15]:
# noting question numbers that need recoding

# multiple choice
q_num_mc = [2,10,11,14,15,18,19,21,24]
q_num_mc = ['q{:02}'.format(num) for num in q_num_mc]
# single choice
q_num_sc = [1,3,16,17,20,22,23,25,27,28]
q_num_sc = ['q{:02}'.format(num) for num in q_num_sc]
# all
q_num_all = q_num_mc + q_num_sc

In [16]:
# replacing '|' with ';'
for q_num in q_num_mc:
    if q_num in pf_fr.columns:
        pf_fr[q_num] = [ans.replace(' | ', ';') for ans in pf_fr[q_num]]
        pf_de[q_num] = [ans.replace(' | ', ';') for ans in pf_de[q_num]]

### recoding

In [17]:
# importing the recoding function
from clean_functions import recode_answers

# regrouping the dataframes
all_pf = [pf_fr, pf_de]

In [18]:
# creating a list of lang variables
langs = ['fr', 'de']

# looping through all datasets
for i, dataset in enumerate(all_pf):
    # setting the right lang var
    lang = langs[i]
    
    # looping through all single choice questions
    for q_num in q_num_sc:
        # skipping questions that are missing
        if q_num in pf_fr.columns:
            # recode and replace
            dataset[q_num] = recode_answers(dataset, q_num, lang, False)
    
    # looping through all multiple choice questions
    for q_num in q_num_mc:
        # skipping questions that are missing
        if q_num in pf_fr.columns:
            # recode and replace
            dataset[q_num] = recode_answers(dataset, q_num, lang, True)

In [19]:
# inspecting the recoding results

cols = ['q{:02}'.format(i+1) for i in range(0,10)]
indices = list(range(30,60))

pf_fr.loc[indices, cols]

,q01,q02,q03,q04,q05,q06,q07,q08,q09,q10
30,C,"[I, L’IA peut rédiger des articles de qualité ...",B,6,7,5,6,4,7,"[C, B, A]"
31,C,"[G, D]",B,7,7,7,7,7,7,"[C, B, A]"
32,C,"[G, E, D]",C,4,4,3,5,4,4,"[B, A]"
33,C,[C],A,6,6,6,6,6,6,[B]
34,D,"[I, G, D, B]",A,4,4,4,4,4,4,[B]
35,D,"[I, D, B]",A,6,7,7,6,7,6,"[C, A]"
36,B,"[E, A]",C,6,6,6,6,6,6,"[D, B, A]"
37,C,"[F, A]",B,3,2,6,6,4,4,[E]
38,C,"[B, A]",B,4,4,5,4,4,4,[E]
39,C,"[I, G, B]",C,5,6,3,6,4,5,"[D, C, A]"


Problems we've detected:
1. q02 --> one answer not properly coded

In [20]:
pf_fr.q02[1], pf_de.q02[1]

(['L’IA peut rédiger des articles de qualité humaine avec un thème clair, une grammaire correcte et un',
  'G',
  'F',
  'E',
  'A'],
 ['KI bleibt ausschließlich in der akademischen Forschung und kann noch nicht kommerziell genutzt werde'])

Clearly PF cut the end of the answer off probably due to its length, we mannually replace

In [21]:
# reecode by hand
to_recode_fr = {pf_fr.q02[1][0]: 'H'}
to_recode_de = {pf_de.q02[1][0]:'B',
'KI kann menschenähnlich Artikel mit einem klaren Thema mit korrekter Grammatik und fortgeschrittenem': 'H',
'Die Architektur eines Recurrent Neural Network wird wahrscheinlich weiterhin an Popularität verliere': 'I'}
pf_fr.q02 = [[to_recode_fr[an] if an in to_recode_fr.keys() else an for an in ans] for ans in pf_fr.q02]
pf_de.q02 = [[to_recode_de[an] if an in to_recode_de.keys() else an for an in ans] for ans in pf_de.q02]

In [22]:
pf_fr.head(3)

,q01,q02,q03,q04,q05,q06,q07,q08,q09,q10,...,q19,q20,q21,q26,q27,q28,q29,country_of_residence,q25_sup,q27_sup
0,D,"[I, G, F, E, D, C]",A,5,6,5,7,5,7,"[D, C, A]",...,"[D, B]",A,"[D, B, A]",Ingénieur métallurgique,B,A,6,FR,18 - 24,high_school
1,D,"[H, G, F, E, A]",A,7,7,6,7,7,7,"[D, A]",...,"[D, A]",A,"[D, C, A]",Ingénieur,D,A,7,FR,18 - 24,high_school
2,B,"[D, B]",C,4,6,4,6,4,6,[E],...,[E],C,[C],Sans profession,A,A,5,FR,45 - 54,vocational_technical_college


### filling in empty columns and concat
let's take a look at the col names

In [23]:
pf_fr.columns

Index(['q01', 'q02', 'q03', 'q04', 'q05', 'q06', 'q07', 'q08', 'q09', 'q10',
       'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20',
       'q21', 'q26', 'q27', 'q28', 'q29', 'country_of_residence', 'q25_sup',
       'q27_sup'],
      dtype='object')

In [24]:
pf_de.columns

Index(['q01', 'q02', 'q03', 'q04', 'q05', 'q06', 'q07', 'q08', 'q09', 'q10',
       'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20',
       'q21', 'q26', 'q27', 'q28', 'q29', 'country_of_residence', 'q25_sup',
       'q27_sup'],
      dtype='object')

In [25]:
# the empty cols
to_placehold = [22,23,24,25]
placeholder_cols = ['q{:02}'.format(i) for i in to_placehold]

In [26]:
# create empty cols
for col in placeholder_cols:
    pf_fr[col] = ['' for _ in range(len(pf_fr))]
    pf_de[col] = ['' for _ in range(len(pf_de))]

Let's recode q27_sup to combine it with q27

In [27]:
# setting up recoding mechanism with a dict
q27_sup_opts = list(set(pf_de.q27_sup))

In [28]:
# recoding dict
recode_vars = ['A', 'E', 'B', 'C', 'A']
q27_sup_opts = {q27_sup_opts[i]:recode_vars[i] for i in range(len(q27_sup_opts))}

In [29]:
# recode fr
pf_fr.q27_sup = [q27_sup_opts[ans] if ans in q27_sup_opts.keys() else ans for ans in pf_fr.q27_sup]

In [30]:
# recode de
pf_de.q27_sup = [q27_sup_opts[ans] if ans in q27_sup_opts.keys() else ans for ans in pf_de.q27_sup]

Now let's see how q27 and q27_sup match up against each other

In [31]:
# comparing q27 & sup
match_q27_fr = [pf_fr.q27[i] == pf_fr.q27_sup[i] for i in range(len(pf_fr))]
match_q27_de = [pf_de.q27[i] == pf_de.q27_sup[i] for i in range(len(pf_fr))]

In [32]:
# percentage of perfect matching
sum(match_q27_fr)/len(match_q27_fr), sum(match_q27_de)/len(match_q27_de)

(0.27, 0.44)

In [33]:
# let's take a look at those who don't match
for i in range(len(pf_fr)):
    print('*******', i, '*******')
    print(pf_fr.q27[i])
    print(pf_fr.q27_sup[i])
    print(' ')

******* 0 *******
B
A
 
******* 1 *******
D
A
 
******* 2 *******
A
B
 
******* 3 *******
B
A
 
******* 4 *******
A
nan
 
******* 5 *******
B
B
 
******* 6 *******
E
C
 
******* 7 *******
B
C
 
******* 8 *******
B
E
 
******* 9 *******
A
A
 
******* 10 *******
A
B
 
******* 11 *******
A
C
 
******* 12 *******
B
A
 
******* 13 *******
C
C
 
******* 14 *******
C
A
 
******* 15 *******
A
B
 
******* 16 *******
B
C
 
******* 17 *******
C
nan
 
******* 18 *******
A
B
 
******* 19 *******
D
A
 
******* 20 *******
A
nan
 
******* 21 *******
A
A
 
******* 22 *******
A
B
 
******* 23 *******
B
C
 
******* 24 *******
B
C
 
******* 25 *******
B
A
 
******* 26 *******
A
B
 
******* 27 *******
B
C
 
******* 28 *******
C
C
 
******* 29 *******
A
B
 
******* 30 *******
B
C
 
******* 31 *******
B
A
 
******* 32 *******
C
C
 
******* 33 *******
D
E
 
******* 34 *******
D
E
 
******* 35 *******
B
C
 
******* 36 *******
B
B
 
******* 37 *******
B
C
 
******* 38 *******
C
C
 
******* 39 *******
B
C
 
****

In [34]:
# let's take a look at those who don't match
for i in range(len(pf_de)):
    print('*******', i, '*******')
    print(pf_de.q27[i])
    print(pf_de.q27_sup[i])
    print(' ')

******* 0 *******
A
B
 
******* 1 *******
Realabschluss
B
 
******* 2 *******
A
A
 
******* 3 *******
realschule
A
 
******* 4 *******
A
A
 
******* 5 *******
A
A
 
******* 6 *******
A
A
 
******* 7 *******
E
A
 
******* 8 *******
A
A
 
******* 9 *******
Realschule
A
 
******* 10 *******
E
A
 
******* 11 *******
A
A
 
******* 12 *******
MSA
A
 
******* 13 *******
A
A
 
******* 14 *******
berufsausbildung
A
 
******* 15 *******
Hauptschule
B
 
******* 16 *******
A
A
 
******* 17 *******
C
C
 
******* 18 *******
A
A
 
******* 19 *******
E
C
 
******* 20 *******
A
A
 
******* 21 *******
Hauptschulabschluss
A
 
******* 22 *******
E
B
 
******* 23 *******
A
A
 
******* 24 *******
Realschule
A
 
******* 25 *******
E
A
 
******* 26 *******
B
B
 
******* 27 *******
Mittlere Reife
A
 
******* 28 *******
A
A
 
******* 29 *******
Fachschule
A
 
******* 30 *******
A
A
 
******* 31 *******
Mittlere Reife
B
 
******* 32 *******
C
C
 
******* 33 *******
E
C
 
******* 34 *******
A
A
 
******* 35 *****

there is too much inconsistency, as a general rule of thumb, we're going to trust our data, and only use the sup data where necessary (custom answers and NaN)

In [35]:
# replacing answers for q27 where appropriate
possible_options = ["A", "B", "C", "D", "E"]
pf_fr.q27 = [ans if ans in possible_options else pf_fr.q27_sup[i] for i, ans in enumerate(pf_fr.q27)]
pf_de.q27 = [ans if ans in possible_options else pf_de.q27_sup[i] for i, ans in enumerate(pf_de.q27)]

### We need to recode the age group too

The categories do not exactly match ours, but they're similar enough, for the sake of the analysis we'll replace them with our categorisation. See the comparison below

             PF            GM
A-----------------[n/a]-----------------[<= 15]

B--------------[18 - 24]--------------[16 - 25]

C--------------[25 - 34]--------------[26 - 35]

D--------------[35 - 44]--------------[36 - 45]

E--------------[45 - 54]--------------[46 - 55]

F----------------[> 54]----------------[>= 56]


As indicated by the chart above, we can convert the data without much loss of accurate information

In [36]:
pf_age_brackets = sorted(list(set(pf_fr.q25_sup)))
pf_age_brackets

['18 - 24', '25 - 34', '35 - 44', '45 - 54', '> 54']

In [37]:
age_codes = ['B', 'C', 'D', 'E', 'F']

In [38]:
# creating recoding dict
age_dict_pf = {pf_age_brackets[i]:age_codes[i] for i in range(len(age_codes))}

In [39]:
# now converting the age column q24
pf_fr.q25 = [age_dict_pf[age] for age in pf_fr.q25_sup]
pf_de.q25 = [age_dict_pf[age] for age in pf_de.q25_sup]

In [40]:
# now we can drop the sup columns since we've encorporated them into the real qs
pf_fr.drop(['q25_sup', 'q27_sup'], axis = 1, inplace = True)
pf_de.drop(['q25_sup', 'q27_sup'], axis = 1, inplace = True)

In [41]:
# reordering the columns
pf_fr = pf_fr[sorted(pf_fr.columns)]
pf_de = pf_de[sorted(pf_fr.columns)]

### Concatenanting and exporting

In [42]:
pf_all = pd.concat([pf_fr, pf_de]).reset_index(drop = True)

In [43]:
pf_all

,country_of_residence,q01,q02,q03,q04,q05,q06,q07,q08,q09,...,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29
0,FR,D,"[I, G, F, E, D, C]",A,5,6,5,7,5,7,...,A,"[D, B, A]",,,,B,Ingénieur métallurgique,B,A,6
1,FR,D,"[H, G, F, E, A]",A,7,7,6,7,7,7,...,A,"[D, C, A]",,,,B,Ingénieur,D,A,7
2,FR,B,"[D, B]",C,4,6,4,6,4,6,...,C,[C],,,,E,Sans profession,A,A,5
3,FR,C,"[G, B]",C,4,3,6,7,4,5,...,C,"[E, D, C, B, A]",,,,B,Etudiante en journalisme,B,A,4
4,BE,A,"[J, C]",C,5,4,5,3,5,6,...,C,"[D, C]",,,,B,Bobo,A,A,6
5,FR,C,[A],B,4,4,4,4,5,4,...,A,[A],,,,F,Restauration publique,B,A,4
6,FR,C,"[H, G, F, E, A]",C,5,6,5,6,4,6,...,C,"[D, C, B, A]",,,,E,Livreur produits fais.,E,A,5
7,FR,C,"[J, I, H]",C,5,5,6,7,3,3,...,C,"[D, B, A]",,,,D,Comptable,B,B,4
8,FR,D,"[I, E, D, B]",C,5,6,4,6,4,5,...,A,"[D, B]",,,,D,Chargé d'études hydrauliques,B,A,3
9,FR,B,[J],B,1,1,1,1,1,1,...,B,"[E, D, C, B, A]",,,,C,Sans,A,A,1


In [44]:
# we will choose to pickle this in order to preserve the list data type
with open(os.path.join(save_dir, 'pf_all.pickle'), 'wb') as handle:
    pickle.dump(pf_all, handle)


# we will also export this as a csv
pf_all.to_csv(os.path.join(save_dir, 'pf_all.csv'))